In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 879.0/879.0 kB 19.2 MB/s eta 0:00:00


In [ ]:
data='drive/My Drive/data_images.zip'

from zipfile import ZipFile
with ZipFile(data,'r') as zip:
    print('Extracting all the files now...')
    zip.extractall()
    print('Done!')

Extracting all the files now...
Done!


In [ ]:
from bs4 import BeautifulSoup
import os
import shutil
import random


pwd = os.getcwd()
source = os.path.join(pwd, 'data_images/images')
os.makedirs('data_images/train/images', exist_ok=True)
os.makedirs('data_images/train/labels', exist_ok=True)
os.makedirs('data_images/val/images', exist_ok=True)
os.makedirs('data_images/val/labels', exist_ok=True)
trainImgDir = os.path.join(pwd, 'data_images/train/images')
trainLabelDir = os.path.join(pwd, 'data_images/train/labels')
valImgDir = os.path.join(pwd, 'data_images/val/images')
valLabelDir = os.path.join(pwd, 'data_images/val/labels')

def moveImgages():
    for dir, _, files in os.walk(source):
        for file in files:
            if file.endswith('jpeg'):
                imgSrcPath = os.path.join(source, file)
                imgDestPath = os.path.join(trainImgDir, file)
                shutil.move(imgSrcPath, imgDestPath)
    print("All images moved !")


def normalizeValues(values):
    xmin = values['xmin']/values['width']
    xmax = values['xmax']/values['width']
    ymin = values['ymin']/values['height']
    ymax = values['ymax']/values['height']

    width = xmax-xmin
    height = ymax-ymin
    center_x = xmin+width/2
    center_y = ymin+height/2

    return center_x, center_y, width, height


def parser():
    tags = ['xmin', 'xmax', 'ymin', 'ymax', 'width', 'height']
    for dir, _, files in os.walk(source):
        for file in files:
            filePath = os.path.join(source, file)
            values = {}
            with open(filePath) as f:
                soup = BeautifulSoup(f, features='lxml-xml')
                for tag in tags:
                    values[tag] = int(soup.find(tag).text)
            center_x, center_y, width, height = normalizeValues(values)
            with open(f'{trainLabelDir}/{file.split(".")[0]}'+'.txt', 'w') as txtFile:
                txtFile.write(f"0 {center_x} {center_y} {width} {height}")


def move_random_data_toVal(trainImgDir,trainLabelDir, valImgDir,valLabelDir, num_files):
    all_files = [f for f in os.listdir( trainImgDir) if os.path.isfile(os.path.join(trainImgDir, f))]

    if num_files > len(all_files):
        num_files = len(all_files)
        print(f"Only {num_files} files available to move.")

    selected_files = random.sample(all_files, num_files)

    for fileName in selected_files:
        imgSrcPath = os.path.join(trainImgDir, fileName)
        imgDestPath = os.path.join(valImgDir, fileName)
        labelName=fileName.split('.')[0]+".txt"
        labelSrcPath = os.path.join(trainLabelDir, labelName)
        labelDestPath = os.path.join(valLabelDir, labelName)
        shutil.move(imgSrcPath, imgDestPath)
        shutil.move(labelSrcPath, labelDestPath)
        print(f"Moved: {fileName} to {valImgDir}")
        print(f"Moved: {labelName} to {valLabelDir}")

def setup():
    moveImgages()
    parser()
    move_random_data_toVal(trainImgDir,trainLabelDir,valImgDir,valLabelDir,25)

setup()

All images moved !
Moved: N85.jpeg to /content/data_images/val/images
Moved: N85.txt to /content/data_images/val/labels
Moved: N20.jpeg to /content/data_images/val/images
Moved: N20.txt to /content/data_images/val/labels
Moved: N220.jpeg to /content/data_images/val/images
Moved: N220.txt to /content/data_images/val/labels
Moved: N73.jpeg to /content/data_images/val/images
Moved: N73.txt to /content/data_images/val/labels
Moved: N169.jpeg to /content/data_images/val/images
Moved: N169.txt to /content/data_images/val/labels
Moved: N218.jpeg to /content/data_images/val/images
Moved: N218.txt to /content/data_images/val/labels
Moved: N179.jpeg to /content/data_images/val/images
Moved: N179.txt to /content/data_images/val/labels
Moved: N121.jpeg to /content/data_images/val/images
Moved: N121.txt to /content/data_images/val/labels
Moved: N162.jpeg to /content/data_images/val/images
Moved: N162.txt to /content/data_images/val/labels
Moved: N143.jpeg to /content/data_images/val/images
Moved: N

In [ ]:
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
!yolo task=detect mode=train model=yolov8l.pt data=./data_images/data.yaml epochs=50 imgsz=640

Ultralytics 8.3.27 🚀 Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8l.pt, data=./data_images/data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train5, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=T

'/content'